In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed

In [2]:
training_data = pickle.load(open("../data/train-v1.1.pkl", "rb"))
print("%d questions in our training set" % len(training_data))

87599 questions in our training set


In [3]:
val_data = pickle.load(open("../data/dev-v1.1.pkl", "rb"))
print("%d questions in our validation set" % len(val_data))

10570 questions in our validation set


In [4]:
word_to_index, index_to_words, word_to_vec_map = pickle.load(open("../embeddings/glove.6B.50d.pkl", "rb"))
vocab_len = len(index_to_words)
print("%d words in our model's vocabulary" % vocab_len)

70056 words in our model's vocabulary


In [5]:
max_qc_len = max([len(qac["question"] + ["<sep>"] + qac["context"]) for qac in training_data])
max_ans_len = max([len(qac["answer"]) + 1 for qac in training_data])

def preprocess(data):
    encoder_input_data = np.zeros((len(data), max_qc_len))
    decoder_input_data = np.zeros((len(data), max_ans_len))
    decoder_target_data = np.zeros((len(data), max_ans_len), dtype=np.uint8)

    for i in range(encoder_input_data.shape[0]):
        qc = data[i]["question"] + ["<sep>"] + data[i]["context"]
        encoder_input_data[i,:] = [word_to_index[word] if word in word_to_index else word_to_index["<unk>"] for word in qc] + [0 for j in range(max_qc_len - len(qc))]

        ans_input = ["<start>"] + data[i]["answer"]
        decoder_input_data[i,:] = [word_to_index[word] if word in word_to_index else word_to_index["<unk>"] for word in ans_input] + [0 for j in range(max_ans_len - len(ans_input))]

        ans_output = data[i]["answer"] + ["<end>"]
        decoder_target_data[i,:] = [word_to_index[word] if word in word_to_index else word_to_index["<unk>"] for word in ans_output] + [0 for j in range(max_ans_len - len(ans_input))]
    
    return encoder_input_data, decoder_input_data, decoder_target_data

encoder_input_data, decoder_input_data, decoder_target_data = preprocess(training_data)
encoder_input_data.shape, decoder_input_data.shape

((87599, 766), (87599, 47))

In [6]:
emb_dim = len(word_to_vec_map['a'])
state_dim = 512
batch_size = 16
epochs = 1
learning_rate = 0.01
path = "saves/keras_LSTMv2_weights.pkl"
load = True

In [7]:
encoder_inputs = Input(shape=(max_qc_len,), dtype='int32')

emb_matrix = np.zeros((vocab_len, emb_dim))
for word, index in word_to_index.items():
    if index != 0 :
        emb_matrix[index, :] = word_to_vec_map[word]
embedding_layer = Embedding(vocab_len, emb_dim, trainable=False, mask_zero=True)
embedding_layer.build((None,))
embedding_layer.set_weights([emb_matrix])

encoder_embeddings = embedding_layer(encoder_inputs)

encoder = LSTM(state_dim, return_state=True, dropout=0.2, recurrent_dropout=0.2)(encoder_embeddings)

encoder_outputs, state_h, state_c = encoder

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(max_ans_len,))
decoder_embeddings = embedding_layer(decoder_inputs)
decoder_lstm = LSTM(state_dim, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)

decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)
outputs = TimeDistributed(Dense(vocab_len, activation='softmax'))(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], outputs)

if load:
    weights = pickle.load(open(path, 'rb'))
    model.set_weights(weights)
model.summary()

W1212 18:06:14.464897 140551604123456 deprecation.py:506] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1212 18:06:15.581856 140551604123456 deprecation.py:506] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1212 18:06:16.293735 140551604123456 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:3794: add_dispatch_support.<locals>.wrapper (from tensorflow.pytho

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 47)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 766)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             3502800     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 512), (None, 1153024     embedding[0][0]              

In [8]:
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')

In [9]:
model.fit([encoder_input_data[:1000], decoder_input_data[:1000]], decoder_target_data[:1000],
          batch_size=batch_size, epochs=epochs)
model.save(path)

1000/1000 [==============================] - 123s 123ms/sample - loss: 0.3060


In [10]:
encoder_input_val, decoder_input_val, decoder_target_val = preprocess(val_data)

In [ ]:
model.evaluate([encoder_input_val, decoder_input_val], decoder_target_val)

10112/10570 [===========================>..] - ETA: 16s - loss: 0.3915